In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Acknowledgment:
# This project benefits significantly from the insights in the GDC (Graph Diffusion Convolution) demo,
# authored by Johannes Klicpera. The notebook—available at
# https://github.com/gasteigerjo/gdc/blob/master/gdc_demo.ipynb
# —served as a valuable reference in developing our own implementation. Our sincere gratitude to the original author.


In [ ]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting
!pip install seaborn


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import scipy.sparse as sp
from collections import defaultdict
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv,GATConv,ChebConv
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, auc,roc_auc_score,roc_curve
import torch.nn as nn
from torch_geometric.utils import to_scipy_sparse_matrix
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cn_fa_feature_path = "/home/snu/Downloads/Histogram_feature_CN_FA_20bin.npy"
mci_fa_feature_path = "/home/snu/Downloads/Histogram_feature_MCI_FA_20bin.npy"

#cn_fa_feature_path = "/content/drive/MyDrive/TejaswiAbburi_va797/Dataset/ISBI_ADNI_CN_dataset/Processed_histogram_features_CN/Histogram_feature_CN_FA_20bin.npy"
#mci_fa_feature_path = "/content/drive/MyDrive/TejaswiAbburi_va797/Dataset/ISBI_ADNI_MCI_dataset/Processed_histogram_features_MCI/Histogram_feature_MCI_FA_20bin.npy"
cn_features_dict = np.load(cn_fa_feature_path, allow_pickle=True).item()
mci_features_dict = np.load(mci_fa_feature_path, allow_pickle=True).item()

cn_features = np.array(list(cn_features_dict.values()))
mci_features = np.array(list(mci_features_dict.values()))

cn_labels = np.zeros(cn_features.shape[0])
mci_labels = np.ones(mci_features.shape[0])

X = np.vstack([cn_features, mci_features])
y = np.hstack([cn_labels, mci_labels])

In [ ]:
X = np.vstack([cn_features, mci_features])
y = np.hstack([cn_labels, mci_labels])

np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]
scaler = StandardScaler()
X = scaler.fit_transform(X)

num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")

Features: (300, 180), Labels: (300,)


In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

In [ ]:
alpha = 0.1
X_norm = F.normalize(X_tensor, p=2, dim=1)
sim_matrix = torch.mm(X_norm, X_norm.T)

src, dst = torch.where(sim_matrix > alpha)
mask = src != dst
src, dst = src[mask], dst[mask]

self_loops = torch.arange(X_tensor.shape[0])
src = torch.cat([src, self_loops])
dst = torch.cat([dst, self_loops])

edges = torch.stack([src, dst], dim=0)
edges_rev = torch.stack([dst, src], dim=0)
edge_index = torch.cat([edges, edges_rev], dim=1)

edge_index = torch.unique(edge_index, dim=1)

print(f"Graph Nodes: {X_tensor.shape[0]}, Edges: {edge_index.shape[1]}")

Graph Nodes: 300, Edges: 16508


In [ ]:
A = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes)

In [ ]:
def gdc(A: sp.csr_matrix, alpha: float, eps: float):
    N = A.shape[0]

    # Self-loops
    A_loop = sp.eye(N) + A

    # Symmetric transition matrix
    D_loop_vec = A_loop.sum(0).A1
    D_loop_vec_invsqrt = 1 / np.sqrt(D_loop_vec)
    D_loop_invsqrt = sp.diags(D_loop_vec_invsqrt)
    T_sym = D_loop_invsqrt @ A_loop @ D_loop_invsqrt

    # PPR-based diffusion
    S = alpha * sp.linalg.inv(sp.eye(N) - (1 - alpha) * T_sym)

    # Sparsify using threshold epsilon
    S_tilde = S.multiply(S >= eps)

    # Column-normalized transition matrix on graph S_tilde
    D_tilde_vec = S_tilde.sum(0).A1
    T_S = S_tilde / D_tilde_vec
    return T_S

In [ ]:
def gdc_heat(A: sp.csr_matrix, t: float, eps: float):
    N = A.shape[0]

    # Self-loops
    A_loop = sp.eye(N) + A

    # Symmetric transition matrix
    D_loop_vec = A_loop.sum(0).A1
    D_loop_vec_invsqrt = 1 / np.sqrt(D_loop_vec)
    D_loop_invsqrt = sp.diags(D_loop_vec_invsqrt)
    T_sym = D_loop_invsqrt @ A_loop @ D_loop_invsqrt

    # Heat-based diffusion
    S = sp.csr_matrix(expm(-t * (np.eye(N) - T_sym)))
    # Sparsify using threshold epsilon
    S_tilde = S.multiply(S >= eps)

    # Column-normalized transition matrix on graph S_tilde
    D_tilde_vec = S_tilde.sum(0).A1
    T_S = S_tilde / D_tilde_vec
    return T_S

In [ ]:
def get_ppr_matrix(
        adj_matrix: np.ndarray,
        alpha: float = 0.1) -> np.ndarray:
    num_nodes = adj_matrix.shape[0]
    A_tilde = adj_matrix + np.eye(num_nodes)
    D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
    H = D_tilde @ A_tilde @ D_tilde
    return alpha * np.linalg.inv(np.eye(num_nodes) - (1 - alpha) * H)

In [ ]:
from scipy.linalg import expm
def get_heat_matrix(
        adj_matrix: np.ndarray,
        t: float = 5.0) -> np.ndarray:
    num_nodes = adj_matrix.shape[0]
    A_tilde = adj_matrix + np.eye(num_nodes)
    D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
    H = D_tilde @ A_tilde @ D_tilde
    return expm(-t * (np.eye(num_nodes) - H))

In [ ]:
def to_edge_index(A_sp):
    return torch.tensor(np.vstack(A_sp.nonzero()), dtype=torch.long)

x_tensor = torch.tensor(X, dtype=torch.float).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data_none = Data(x=x_tensor, edge_index=edge_index.to(device), y=y_tensor)
A_heat = gdc_heat(A, t=2.0, eps=0.1)
A_ppr = gdc(A, alpha=0.15, eps=0.1)
# A_heat = get_heat_matrix(A.toarray(),t=5.0)
# A_ppr = get_ppr_matrix(A.toarray(),alpha=0.05)
data_heat = Data(x=x_tensor, edge_index=to_edge_index(A_heat).to(device), y=y_tensor)
data_ppr = Data(x=x_tensor, edge_index=to_edge_index(A_ppr).to(device), y=y_tensor)

In [ ]:
class Cheb(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, K=3, dropout=0.2):
        super().__init__()
        self.conv1 = ChebConv(in_dim, hidden_dim, K=K)
        self.conv2 = ChebConv(hidden_dim, out_dim, K=K)
        self.dropout = dropout

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
graph_datasets = {
    "none": data_none,
    "heat": data_heat,
    "ppr": data_ppr
}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=42)
num_epochs = 5000

for name, data in graph_datasets.items():
    print(f"\n\n===== Running 20-Fold CV for {name.upper()} Graph =====")
    accuracies, precisions, recalls, f1_scores, losses = [], [], [], [], []
    all_fpr, all_tpr, all_auc = [], [], []
    all_y_true, all_y_proba = [], []

    data = data.to(device)
    num_feats = data.num_node_features

    for fold, (train_idx, test_idx) in enumerate(sss.split(X, y)):
        print(f"\nTraining fold {fold + 1} for {name} graph")

        train_mask = torch.tensor(train_idx, dtype=torch.long, device=device)
        test_mask  = torch.tensor(test_idx,  dtype=torch.long, device=device)

        model = Cheb(num_feats, 512, 2, dropout=0.2).to(device)
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)
        loss_fn = nn.NLLLoss()


        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = loss_fn(out[train_mask], y_tensor[train_mask])
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            logits    = model(data)
            y_pred = logits.argmax(dim=1).cpu().numpy()
            y_proba = torch.exp(logits).cpu().numpy()[:, 1]  # Probabilities for class 1

        y_true_test = y_tensor[test_mask].cpu().numpy()
        y_pred_test = y_pred[test_mask.cpu().numpy()]
        y_proba_test = y_proba[test_mask.cpu().numpy()]

        acc = accuracy_score(y_true_test, y_pred_test)
        prec = precision_score(y_true_test, y_pred_test)
        rec = recall_score(y_true_test, y_pred_test)
        f1 = f1_score(y_true_test, y_pred_test)
        ce_loss = log_loss(y_true_test, y_proba_test)
        auc = roc_auc_score(y_true_test, y_proba_test)
        fpr, tpr, _ = roc_curve(y_true_test, y_proba_test)


        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1_scores.append(f1)
        losses.append(ce_loss)
        all_auc.append(auc)
        all_fpr.append(fpr)
        all_tpr.append(tpr)
        all_y_true.extend(y_true_test)
        all_y_proba.extend(y_proba_test)

        print(f"Fold {fold + 1}: Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
              f"F1={f1:.4f} | Loss={ce_loss:.4f} | AUC={auc:.4f}")

    # Final aggregated metrics with mean and std
    print(f"\n==== {name.upper()} Graph 20-Fold Average Results ====")
    print(f"Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"Precision: {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
    print(f"Recall: {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
    print(f"F1 Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
    print(f"Cross-Entropy Loss: {np.mean(losses):.4f} ± {np.std(losses):.4f}")
    print(f"AUC: {np.mean(all_auc):.4f} ± {np.std(all_auc):.4f}")



===== Running 20-Fold CV for NONE Graph =====

Training fold 1 for none graph
Fold 1: Acc=0.7000 | Prec=0.6971 | Rec=0.8133 | F1=0.7508 | Loss=0.7142 | AUC=0.7726

Training fold 2 for none graph
Fold 2: Acc=0.6852 | Prec=0.6879 | Rec=0.7933 | F1=0.7368 | Loss=0.7526 | AUC=0.7149

Training fold 3 for none graph
Fold 3: Acc=0.6593 | Prec=0.6648 | Rec=0.7800 | F1=0.7178 | Loss=0.7714 | AUC=0.7234

Training fold 4 for none graph
Fold 4: Acc=0.6741 | Prec=0.7348 | Rec=0.6467 | F1=0.6879 | Loss=0.6863 | AUC=0.7598

Training fold 5 for none graph
Fold 5: Acc=0.7259 | Prec=0.7235 | Rec=0.8200 | F1=0.7688 | Loss=0.9080 | AUC=0.7364

Training fold 6 for none graph
Fold 6: Acc=0.6630 | Prec=0.7153 | Rec=0.6533 | F1=0.6829 | Loss=0.8017 | AUC=0.7085

Training fold 7 for none graph
Fold 7: Acc=0.7185 | Prec=0.7256 | Rec=0.7933 | F1=0.7580 | Loss=0.7036 | AUC=0.7645

Training fold 8 for none graph
Fold 8: Acc=0.7259 | Prec=0.7135 | Rec=0.8467 | F1=0.7744 | Loss=0.6852 | AUC=0.7631

Training fold 9

In [ ]:
plt.figure(figsize=(8, 6))

for i in range(len(all_fpr)):
    plt.plot(all_fpr[i], all_tpr[i], alpha=0.3, label=f"Fold {i+1} AUC = {all_auc[i]:.2f}")

fpr_avg, tpr_avg, _ = roc_curve(all_y_true, all_y_proba)
auc_avg = roc_auc_score(all_y_true, all_y_proba)
plt.plot(fpr_avg, tpr_avg, color='blue', lw=2, label=f"Mean ROC (AUC = {auc_avg:.2f})")

plt.plot([0, 1], [0, 1], 'k--', label="Chance")

plt.xlabel("False Positive Rate", fontsize=14)
plt.ylabel("True Positive Rate", fontsize=14)
plt.title("ROC Curve Across 20 Folds (GDC-Chebconv)", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(loc="lower right", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.show()

When alpha = 0

===== Running 20-Fold CV for NONE Graph =====

Training fold 1 for none graph
Fold 1: Acc=0.6444 | Prec=0.6849 | Rec=0.6667 | F1=0.6757 | Loss=1.0262

Training fold 2 for none graph
Fold 2: Acc=0.7630 | Prec=0.7756 | Rec=0.8067 | F1=0.7908 | Loss=0.5848

Training fold 3 for none graph
Fold 3: Acc=0.6630 | Prec=0.7323 | Rec=0.6200 | F1=0.6715 | Loss=1.0431

Training fold 4 for none graph
Fold 4: Acc=0.7148 | Prec=0.7355 | Rec=0.7600 | F1=0.7475 | Loss=1.2836

Training fold 5 for none graph
Fold 5: Acc=0.7296 | Prec=0.7225 | Rec=0.8333 | F1=0.7740 | Loss=2.0795

Training fold 6 for none graph
Fold 6: Acc=0.6889 | Prec=0.7845 | Rec=0.6067 | F1=0.6842 | Loss=0.9347

Training fold 7 for none graph
Fold 7: Acc=0.7148 | Prec=0.7920 | Rec=0.6600 | F1=0.7200 | Loss=1.0443

Training fold 8 for none graph
Fold 8: Acc=0.7630 | Prec=0.7500 | Rec=0.8600 | F1=0.8012 | Loss=0.7468

Training fold 9 for none graph
Fold 9: Acc=0.6630 | Prec=0.7323 | Rec=0.6200 | F1=0.6715 | Loss=1.2105

Training fold 10 for none graph
Fold 10: Acc=0.7185 | Prec=0.7229 | Rec=0.8000 | F1=0.7595 | Loss=0.9980

Training fold 11 for none graph
Fold 11: Acc=0.7259 | Prec=0.7533 | Rec=0.7533 | F1=0.7533 | Loss=1.0229

Training fold 12 for none graph
Fold 12: Acc=0.7259 | Prec=0.7209 | Rec=0.8267 | F1=0.7702 | Loss=0.9860

Training fold 13 for none graph
Fold 13: Acc=0.7556 | Prec=0.7283 | Rec=0.8933 | F1=0.8024 | Loss=1.0285

Training fold 14 for none graph
Fold 14: Acc=0.6741 | Prec=0.7719 | Rec=0.5867 | F1=0.6667 | Loss=1.1702

Training fold 15 for none graph
Fold 15: Acc=0.7370 | Prec=0.7423 | Rec=0.8067 | F1=0.7732 | Loss=1.0418

Training fold 16 for none graph
Fold 16: Acc=0.7296 | Prec=0.7810 | Rec=0.7133 | F1=0.7456 | Loss=0.8196

Training fold 17 for none graph
Fold 17: Acc=0.7333 | Prec=0.7468 | Rec=0.7867 | F1=0.7662 | Loss=0.7774

Training fold 18 for none graph
Fold 18: Acc=0.7333 | Prec=0.7600 | Rec=0.7600 | F1=0.7600 | Loss=1.2302

Training fold 19 for none graph
Fold 19: Acc=0.6741 | Prec=0.6867 | Rec=0.7600 | F1=0.7215 | Loss=1.1280

Training fold 20 for none graph
Fold 20: Acc=0.6815 | Prec=0.7254 | Rec=0.6867 | F1=0.7055 | Loss=0.7750

==== NONE Graph 20-Fold Average Results ====
Avg Acc=0.7117 | Avg Prec=0.7425 | Avg Rec=0.7403 | Avg F1=0.7380 | Avg Loss=1.0466


===== Running 20-Fold CV for HEAT Graph =====

Training fold 1 for heat graph
Fold 1: Acc=0.7444 | Prec=0.7793 | Rec=0.7533 | F1=0.7661 | Loss=0.6634

Training fold 2 for heat graph
Fold 2: Acc=0.7148 | Prec=0.7355 | Rec=0.7600 | F1=0.7475 | Loss=0.7208

Training fold 3 for heat graph
Fold 3: Acc=0.6926 | Prec=0.7343 | Rec=0.7000 | F1=0.7167 | Loss=0.7711

Training fold 4 for heat graph
Fold 4: Acc=0.7519 | Prec=0.7862 | Rec=0.7600 | F1=0.7729 | Loss=0.6621

Training fold 5 for heat graph
Fold 5: Acc=0.7296 | Prec=0.7333 | Rec=0.8067 | F1=0.7683 | Loss=1.0293

Training fold 6 for heat graph
Fold 6: Acc=0.7037 | Prec=0.7652 | Rec=0.6733 | F1=0.7163 | Loss=0.7929

Training fold 7 for heat graph
Fold 7: Acc=0.7074 | Prec=0.7483 | Rec=0.7133 | F1=0.7304 | Loss=0.7919

Training fold 8 for heat graph
Fold 8: Acc=0.7667 | Prec=0.7605 | Rec=0.8467 | F1=0.8013 | Loss=0.6479

Training fold 9 for heat graph
Fold 9: Acc=0.6926 | Prec=0.7597 | Rec=0.6533 | F1=0.7025 | Loss=0.9527

Training fold 10 for heat graph
Fold 10: Acc=0.7333 | Prec=0.7378 | Rec=0.8067 | F1=0.7707 | Loss=0.7733

Training fold 11 for heat graph
Fold 11: Acc=0.7185 | Prec=0.7284 | Rec=0.7867 | F1=0.7564 | Loss=0.6970

Training fold 12 for heat graph
Fold 12: Acc=0.7556 | Prec=0.7658 | Rec=0.8067 | F1=0.7857 | Loss=0.6952

Training fold 13 for heat graph
Fold 13: Acc=0.7148 | Prec=0.7325 | Rec=0.7667 | F1=0.7492 | Loss=0.8044

Training fold 14 for heat graph
Fold 14: Acc=0.6630 | Prec=0.7092 | Rec=0.6667 | F1=0.6873 | Loss=0.9734

Training fold 15 for heat graph
Fold 15: Acc=0.7741 | Prec=0.7947 | Rec=0.8000 | F1=0.7973 | Loss=0.6749

Training fold 16 for heat graph
Fold 16: Acc=0.7815 | Prec=0.7862 | Rec=0.8333 | F1=0.8091 | Loss=0.7412

Training fold 17 for heat graph
Fold 17: Acc=0.6963 | Prec=0.7237 | Rec=0.7333 | F1=0.7285 | Loss=0.8609

Training fold 18 for heat graph
Fold 18: Acc=0.7111 | Prec=0.7500 | Rec=0.7200 | F1=0.7347 | Loss=0.7224

Training fold 19 for heat graph
Fold 19: Acc=0.7111 | Prec=0.7195 | Rec=0.7867 | F1=0.7516 | Loss=0.9156

Training fold 20 for heat graph
Fold 20: Acc=0.7074 | Prec=0.7178 | Rec=0.7800 | F1=0.7476 | Loss=0.7517

==== HEAT Graph 20-Fold Average Results ====
Avg Acc=0.7235 | Avg Prec=0.7484 | Avg Rec=0.7577 | Avg F1=0.7520 | Avg Loss=0.7821


===== Running 20-Fold CV for PPR Graph =====

Training fold 1 for ppr graph
Fold 1: Acc=0.7407 | Prec=0.7667 | Rec=0.7667 | F1=0.7667 | Loss=0.6939

Training fold 2 for ppr graph
Fold 2: Acc=0.7185 | Prec=0.7500 | Rec=0.7400 | F1=0.7450 | Loss=0.7584

Training fold 3 for ppr graph
Fold 3: Acc=0.6889 | Prec=0.7200 | Rec=0.7200 | F1=0.7200 | Loss=0.7658

Training fold 4 for ppr graph
Fold 4: Acc=0.7630 | Prec=0.7945 | Rec=0.7733 | F1=0.7838 | Loss=0.6519

Training fold 5 for ppr graph
Fold 5: Acc=0.7370 | Prec=0.7310 | Rec=0.8333 | F1=0.7788 | Loss=1.0700

Training fold 6 for ppr graph
Fold 6: Acc=0.6704 | Prec=0.7521 | Rec=0.6067 | F1=0.6716 | Loss=0.8375

Training fold 7 for ppr graph
Fold 7: Acc=0.7074 | Prec=0.7554 | Rec=0.7000 | F1=0.7266 | Loss=0.7799

Training fold 8 for ppr graph
Fold 8: Acc=0.7630 | Prec=0.7560 | Rec=0.8467 | F1=0.7987 | Loss=0.6454

Training fold 9 for ppr graph
Fold 9: Acc=0.6481 | Prec=0.6923 | Rec=0.6600 | F1=0.6758 | Loss=0.9620

Training fold 10 for ppr graph
Fold 10: Acc=0.7222 | Prec=0.7517 | Rec=0.7467 | F1=0.7492 | Loss=0.7410

Training fold 11 for ppr graph
Fold 11: Acc=0.7000 | Prec=0.7518 | Rec=0.6867 | F1=0.7178 | Loss=0.7213

Training fold 12 for ppr graph
Fold 12: Acc=0.7222 | Prec=0.7778 | Rec=0.7000 | F1=0.7368 | Loss=0.7088

Training fold 13 for ppr graph
Fold 13: Acc=0.7407 | Prec=0.7410 | Rec=0.8200 | F1=0.7785 | Loss=0.7892

Training fold 14 for ppr graph
Fold 14: Acc=0.6630 | Prec=0.7092 | Rec=0.6667 | F1=0.6873 | Loss=0.9339

Training fold 15 for ppr graph
Fold 15: Acc=0.7741 | Prec=0.7908 | Rec=0.8067 | F1=0.7987 | Loss=0.6581

Training fold 16 for ppr graph
Fold 16: Acc=0.7889 | Prec=0.8000 | Rec=0.8267 | F1=0.8131 | Loss=0.7573

Training fold 17 for ppr graph
Fold 17: Acc=0.6852 | Prec=0.7211 | Rec=0.7067 | F1=0.7138 | Loss=0.8338

Training fold 18 for ppr graph
Fold 18: Acc=0.7000 | Prec=0.7315 | Rec=0.7267 | F1=0.7291 | Loss=0.7684

Training fold 19 for ppr graph
Fold 19: Acc=0.6926 | Prec=0.7055 | Rec=0.7667 | F1=0.7348 | Loss=0.8800

Training fold 20 for ppr graph
Fold 20: Acc=0.6963 | Prec=0.7267 | Rec=0.7267 | F1=0.7267 | Loss=0.7200

==== PPR Graph 20-Fold Average Results ====
Avg Acc=0.7161 | Avg Prec=0.7463 | Avg Rec=0.7413 | Avg F1=0.7426 | Avg Loss=0.7838

In [ ]:
# results = {}
# for name, d in zip(['none', 'heat', 'ppr'], [data_none, data_heat, data_ppr]):
#     results[name] = train_eval(d, seeds=50)

# for name, best_dict in results.items():
#     boots_series = sns.algorithms.bootstrap(best_dict['test_acc'], func=np.mean, n_boot=1000)
#     ci = np.max(np.abs(sns.utils.ci(boots_series, 95) - np.mean(best_dict['test_acc'])))
#     mean_acc = np.mean(best_dict['test_acc'])
#     print(f"{name}: Mean accuracy: {100*mean_acc:.2f} ± {100*ci:.2f}%")